In [23]:
import pybamm
import scipy
import numpy as np

model = pybamm.lithium_ion.SPMe()
model2 = pybamm.lithium_ion.SPMe()
param = pybamm.ParameterValues("Chen2020")
param2 = pybamm.ParameterValues("Chen2020")

param2["Negative particle diffusivity [m2.s-1]"] = 5e-14
param2["Positive particle diffusivity [m2.s-1]"] = 5e-14

experiment = pybamm.Experiment(
    [
        "Discharge at 1C until 3.3V",
    ],
    period="1 seconds",
)


def objective_function(x):
    # param.({"Negative electrode diffusivity [m2.s-1]": x[0], "Positive electrode diffusivity [m2.s-1]": x[1]})
    param["Negative particle diffusivity [m2.s-1]"] = x[0]
    param["Positive particle diffusivity [m2.s-1]"] = x[1]
    sim = pybamm.Simulation(model, experiment=experiment, parameter_values=param)
    sim2 = pybamm.Simulation(model2, experiment=experiment, parameter_values=param2)
    solution = sim.solve()
    solution2 = sim2.solve()
    error = np.sum(
        (solution2["Terminal voltage [V]"].data - solution["Terminal voltage [V]"].data)
        ** 2
    )
    return error


initial_guess = [2e-10, 2e-10]

result = scipy.optimize.minimize(objective_function, initial_guess)

optimized_params = result.x

print("Optimized parameters:", optimized_params)

ValueError: operands could not be broadcast together with shapes (2990,) (3031,) 

In [12]:
solution = sim.solve()
solution2 = sim2.solve()

2025-02-05 12:48:29.971 - [ERROR] callbacks.on_experiment_error(233): Simulation error: Events ['Minimum voltage [V]'] are non-positive at initial conditions
2025-02-05 12:48:30.189 - [ERROR] callbacks.on_experiment_error(233): Simulation error: Events ['Minimum voltage [V]'] are non-positive at initial conditions


In [13]:
np.shape(solution["Terminal voltage [V]"].data)

(16853,)

In [14]:
np.shape(solution2["Terminal voltage [V]"].data)

(26063,)